In [ ]:
import pandas as pd
data = pd.read_csv('/content/enhanced_combined_final3.csv')

In [ ]:
# Assuming the dataset has the following columns: 'EDA', 'HR', 'Temp', and 'Stress_Level'
# Split the data into features and labels
X = data.drop('Stress_Level', axis=1)
y = data['Stress_Level']

# Normalize/Standardize the features (you can use MinMaxScaler or StandardScaler from sklearn.preprocessing)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

# Split the temp set into validation and testing sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.1, random_state=42)


In [ ]:
# Reshape input data to [samples, timesteps, features]
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val_reshaped = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# One-hot encode labels
from tensorflow.keras.utils import to_categorical

y_train_categorical = to_categorical(y_train)
y_val_categorical = to_categorical(y_val)
y_test_categorical = to_categorical(y_test)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(1, X_train.shape[1])))
model.add(Dropout(0.9))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define ModelCheckpoint to save the best model
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# Define EarlyStopping to stop training if there's no improvement in the validation loss
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)

# Train the model with callbacks
history = model.fit(
    X_train_reshaped,
    y_train_categorical,
    epochs=20,
    batch_size=64,
    validation_data=(X_val_reshaped, y_val_categorical),
    callbacks=[checkpoint, early_stopping],  # Use the callbacks parameter to include both callbacks
    verbose=1
)

# Evaluate the model on the test set
evaluation = model.evaluate(X_test_reshaped, y_test_categorical, verbose=1)
print(f'Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}')


Epoch 1/20
284/291 [============================>.] - ETA: 0s - loss: 0.7369 - accuracy: 0.7007
Epoch 1: val_loss improved from inf to 0.42287, saving model to best_model.h5
291/291 [==============================] - 4s 5ms/step - loss: 0.7317 - accuracy: 0.7034 - val_loss: 0.4229 - val_accuracy: 0.8726
Epoch 2/20
 55/291 [====>.........................] - ETA: 0s - loss: 0.5113 - accuracy: 0.8009

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


291/291 [==============================] - ETA: 0s - loss: 0.4455 - accuracy: 0.8177
Epoch 2: val_loss improved from 0.42287 to 0.27438, saving model to best_model.h5
291/291 [==============================] - 1s 3ms/step - loss: 0.4455 - accuracy: 0.8177 - val_loss: 0.2744 - val_accuracy: 0.8759
Epoch 3/20
291/291 [==============================] - ETA: 0s - loss: 0.3601 - accuracy: 0.8402
Epoch 3: val_loss improved from 0.27438 to 0.22392, saving model to best_model.h5
291/291 [==============================] - 1s 3ms/step - loss: 0.3601 - accuracy: 0.8402 - val_loss: 0.2239 - val_accuracy: 0.8866
Epoch 4/20
282/291 [============================>.] - ETA: 0s - loss: 0.3218 - accuracy: 0.8530
Epoch 4: val_loss improved from 0.22392 to 0.19925, saving model to best_model.h5
291/291 [==============================] - 1s 3ms/step - loss: 0.3211 - accuracy: 0.8534 - val_loss: 0.1992 - val_accuracy: 0.9124
Epoch 5/20
274/291 [===========================>..] - ETA: 0s - loss: 0.2984 - accur

In [ ]:
# Save the model
model.save('my_stress_model.h5')

# Download the model file
from google.colab import files
files.download('my_stress_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>